In [14]:
import requests
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Descargar archivo PeeringDB

In [ ]:
def dowload_peeringdb(year:str='2022',month:str='01',day:str='01'):
    """
    Download the latest PeeringDB data from the official website.
    """

    # URL de descarga de PeeringDB data en caida
    url = f"https://publicdata.caida.org/datasets/peeringdb/{year}/{month}/peeringdb_2_dump_{year}_{month}_{day}.json"

    # Carpeta donde se guarda el archivo
    file_path = os.path.join('data/peeringdb', "peeringdb.json")

    # Descargar el archivo
    try:
        response = requests.get(url)
    except:
        print(f"Error downloading PeeringDB data: {e}")
        return

    with open(file_path, "wb") as file:
        file.write(response.content)

    print(f"Downloaded PeeringDB data to {file_path}")

In [4]:
year = '2022'
month = '07'
day = '01'

dowload_peeringdb(year, month, day)

Downloaded PeeringDB data to data/peeringdb/peeringdb.json


# Archivo de PeeringDB
* ixlan :
* ixfac :
* ix :
* as_set :
* api :
* fac :
* netfac :
* org :
* net : Redes (ASes) registradas en PeeringDB.
* netixlan :
* poc :
* ixpfx :

Cada Sistema Autonomo registrado tendra la siguiente Informacion:

📋 Información general
* asn : Número de Sistema Autónomo
* name : Nombre corto de la red u organización.
* name_long : Nombre largo o completo de la organización.
* aka : Otros nombres por los que se conoce a la organización (alias).
* **org_id** : ID de la organización dentro de PeeringDB.
* website : Sitio web
* notes : Notas adicionales que la red ha dejado públicas.


🌐 Infraestructura de red

* **fac_count** : Número de datacenters (facilidades) donde opera.
* **ix_count** : Número de IXPs (puntos de intercambio de tráfico) en los que participa.
* netfac_updated : Fecha de última actualización de datos en facilidades.
* netixlan_updated : Fecha de última actualización de datos en IXPs.

🔄 Políticas de peering

* **policy_general** : Política de peering: puede ser {'Open', '', 'Selective', 'No', 'Restrictive'}.
* **policy_ratio** : Ratio de tráfico que influye en si se acepta peering  {False, True}.
* **policy_locations** : Lugares donde aplica la política de peering. {'', 'Required - US', 'Preferred', 'Required - EU', 'Required - International', 'Not Required'}
* **policy_contracts** : Si se requiere contrato para establecer peering. {'', 'Private Only', 'Not Required', 'Required'}
* policy_url : URL pública donde se detalla la política de peering.


📶 Capacidad y tipo de tráfico
* **info_prefixes4** : Cantidad de prefijos IPv4 que anuncia la red.
* **info_prefixes6** : Cantidad de prefijos IPv6 que anuncia la red.
* **info_traffic** : Nivel estimado de tráfico (ej: 100-200Gbps, 20Gbps, etc.).Unique values in : {'', '100-200Gbps', '10-20Tbps', '100-1000Mbps', '50-100Gbps', '500-1000Gbps', '50-100Tbps', '100+Tbps', '5-10Gbps', '200-300Gbps', '0-20Mbps', '20-50Tbps', '1-5Gbps', '20-50Gbps', '1-5Tbps', '300-500Gbps', '5-10Tbps', '20-100Mbps', '10-20Gbps'}
* **info_unicast** : Si soporta tráfico unicast (la mayoría sí). {False, True}
* **info_multicast** : Si soporta tráfico multicast. {False, True}
* **info_ipv6** : Si soporta tráfico IPv6. {False, True}
* **info_scope** : Alcance de la red: puede ser "Global", "Regional", "National", etc. Unique values in : {'', 'Not Disclosed', 'Asia Pacific', 'Europe', 'Regional', 'Middle East', 'North America', 'Africa', 'Global', 'South America', 'Australia'}
* **info_type** : Tipo de red: "NSP" (network service provider), "Content", "Enterprise", etc. Unique values in : {'', 'Content', 'Enterprise', 'Not Disclosed', 'Route Server', 'Educational/Research', 'Government', 'Non-Profit', 'Network Services', 'Cable/DSL/ISP', 'NSP', 'Route Collector'}
* **info_never_via_route_servers** : Indica si no acepta rutas a través de route servers.

* **info_ratio** : {'', 'Balanced', 'Not Disclosed', 'Mostly Inbound', 'Mostly Outbound', 'Heavy Inbound', 'Heavy Outbound'}
🔍 Monitoreo y herramientas

* route_server 
* looking_glass : URL de su looking glass (para ver rutas que anuncia o recibe).
* status_dashboard : Página donde publica el estado de su red (caídas, mantenimientos, etc.).


📅 Tiempos y actualizaciones

* created : Fecha de creación del registro en PeeringDB.
* updated : Última fecha de actualización del registro general. 
* poc_updated : Última vez que se actualizaron los contactos (Points of Contact).
* id  :
* allow_ixp_update : Indica si los IXPs pueden actualizar la información de esta red.
* irr_as_set : Set de objetos IRR (Internet Routing Registry) usados para filtrado de rutas.

In [16]:
import json

peeringdb_file = 'data/peeringdb/peeringdb.json'
with open(peeringdb_file) as f:
    data = json.load(f)['net']['data']
    first_as = data[0]
    print(first_as.keys())

dict_keys(['status', 'looking_glass', 'route_server', 'netixlan_updated', 'info_ratio', 'id', 'policy_locations', 'policy_ratio', 'status_dashboard', 'info_unicast', 'created', 'name_long', 'policy_general', 'website', 'allow_ixp_update', 'updated', 'netfac_updated', 'info_traffic', 'info_multicast', 'asn', 'info_prefixes4', 'name', 'info_scope', 'notes', 'ix_count', 'org_id', 'policy_url', 'info_never_via_route_servers', 'poc_updated', 'info_type', 'policy_contracts', 'info_prefixes6', 'aka', 'fac_count', 'info_ipv6', 'irr_as_set'])


Creo un diccionario con cada atributos que me indique Si el atributo es :
* Categorico o numerico
* Si es categorico se transforma en un one-hot vector

In [17]:
# Listas con los nombres de los atributos que nos interesan
num_list = ['ix_count', 'fac_count', 'info_prefixes4', 'info_prefixes6']
categorical_list = ['policy_general', 'policy_locations', 'policy_contracts', 'info_traffic', 'info_scope', 'info_type', 'info_ratio']
boolean_list = ['policy_ratio', 'info_unicast', 'info_multicast', 'info_ipv6', 'info_never_via_route_servers']

# Lista combinada de todos los atributos
list_all = num_list + categorical_list + boolean_list

# Inicializo un diccionario vacío
dict_attr = {}

# Recorrer cada atributo de la lista
for attr in list_all:
    # Asegurarse de que cada atributo tenga un diccionario inicializado
    if attr not in dict_attr:
        dict_attr[attr] = {}

    # Determinar si es numérico o no y agregarlo al diccionario
    if attr in num_list:
        dict_attr[attr]['categorical'] = False
    else:
        dict_attr[attr]['categorical'] = True

    # Set para los valores únicos
    uniques = set()
    count_empty = 0  # Inicializo el contador de valores vacíos

    # Iterar sobre los datos de 'data'
    for asn in data:
        value = asn.get(attr)  # Usar .get() para manejar posibles claves faltantes

        # Contar valores vacíos
        if value == '' or value is None:
            count_empty += 1

        # Agregar el valor al set de valores únicos
        uniques.add(value)

    # Guardar el número de valores únicos
    dict_attr[attr]['uniques'] = len(uniques)
    dict_attr[attr]['empty_count'] = count_empty  # Guardar el conteo de valores vacíos si es necesario

# Mostrar el diccionario resultante
for key, value in dict_attr.items():
    print(f"{key}: {value}")



ix_count: {'categorical': False, 'uniques': 83, 'empty_count': 0}
fac_count: {'categorical': False, 'uniques': 91, 'empty_count': 0}
info_prefixes4: {'categorical': False, 'uniques': 364, 'empty_count': 0}
info_prefixes6: {'categorical': False, 'uniques': 188, 'empty_count': 0}
policy_general: {'categorical': True, 'uniques': 5, 'empty_count': 1080}
policy_locations: {'categorical': True, 'uniques': 6, 'empty_count': 1080}
policy_contracts: {'categorical': True, 'uniques': 4, 'empty_count': 1212}
info_traffic: {'categorical': True, 'uniques': 19, 'empty_count': 10217}
info_scope: {'categorical': True, 'uniques': 11, 'empty_count': 257}
info_type: {'categorical': True, 'uniques': 12, 'empty_count': 336}
info_ratio: {'categorical': True, 'uniques': 7, 'empty_count': 492}
policy_ratio: {'categorical': True, 'uniques': 2, 'empty_count': 0}
info_unicast: {'categorical': True, 'uniques': 2, 'empty_count': 0}
info_multicast: {'categorical': True, 'uniques': 2, 'empty_count': 0}
info_ipv6: {'c

In [18]:
import pandas as pd

# Listas de atributos
num_list = ['asn','org_id', 'ix_count', 'fac_count', 'info_prefixes4', 'info_prefixes6']
categorical_list = ['policy_general', 'policy_locations', 'policy_contracts', 'info_traffic', 'info_scope', 'info_type', 'info_ratio']
boolean_list = ['policy_ratio', 'info_unicast', 'info_multicast', 'info_ipv6', 'info_never_via_route_servers']
list_all = num_list + categorical_list + boolean_list

# Crear DataFrame con los datos numéricos
num_data = {attr: [asn.get(attr, None) for asn in data] for attr in num_list}
num_df = pd.DataFrame(num_data)

# Crear DataFrame con los datos categóricos y hacer one-hot encoding
cat_data = {attr: [asn.get(attr, None) for asn in data] for attr in categorical_list}
cat_df = pd.DataFrame(cat_data)

# Realizar One-Hot Encoding para los atributos categóricos
cat_df_encoded = pd.get_dummies(cat_df, columns=categorical_list, prefix=categorical_list)

# Crear DataFrame para los valores booleanos
bool_data = {attr: [asn.get(attr, False) for asn in data] for attr in boolean_list}
bool_df = pd.DataFrame(bool_data)

# Convertir los valores booleanos de True/False a 1/0
bool_df = bool_df.applymap(lambda x: 1 if x else 0)

# Concatenar todos los DataFrames (numéricos, categóricos y booleanos)
final_df = pd.concat([num_df, cat_df_encoded, bool_df], axis=1)

# Mostrar el DataFrame final
print(final_df)



          asn  org_id  ix_count  fac_count  info_prefixes4  info_prefixes6  \
0        4436    8897         0          0          200000           10000   
1       20940      14       188        175           12000            5000   
2       31800      17        15          0               3               0   
3       22822      18        93         72            5000             500   
4        3303    9350        54         32           10000            1800   
...       ...     ...       ...        ...             ...             ...   
25268  149835   32741         0          0               0              30   
25269    1025   31600         0          0               0               0   
25270  137113   32742         0          0              40              80   
25271  269626   32743         0          0               0               0   
25272  267550   32744         0          0               0               0   

       policy_general_  policy_general_No  policy_general_Open 

/tmp/ipykernel_7766/1275927829.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  bool_df = bool_df.applymap(lambda x: 1 if x else 0)


In [19]:
# Contar el número de columnas
num_columns = final_df.shape[1]

# Mostrar el número de columnas
print(f'Número de columnas: {num_columns}')


Número de columnas: 75


# Reducir varianza de atributros numericos


In [22]:
num_list = ['asn','org_id', 'ix_count', 'fac_count', 'info_prefixes4', 'info_prefixes6']
num_list = num_list[1:]

In [23]:
# Reducir la varianza

for col in num_list:
    final_df[col] = np.log1p(final_df[col] + 1)
final_df

,asn,org_id,ix_count,fac_count,info_prefixes4,info_prefixes6,policy_general_,policy_general_No,policy_general_Open,policy_general_Restrictive,...,info_ratio_Heavy Inbound,info_ratio_Heavy Outbound,info_ratio_Mostly Inbound,info_ratio_Mostly Outbound,info_ratio_Not Disclosed,policy_ratio,info_unicast,info_multicast,info_ipv6,info_never_via_route_servers
0,4436,9.093694,0.693147,0.693147,12.206083,9.210540,False,False,False,True,...,False,False,False,False,False,1,1,0,1,0
1,20940,2.772589,5.247024,5.176150,9.392829,8.517593,False,False,True,False,...,False,True,False,False,False,0,1,0,1,0
2,31800,2.944439,2.833213,0.693147,1.609438,0.693147,False,False,True,False,...,True,False,False,False,False,0,1,0,0,0
3,22822,2.995732,4.553877,4.304065,8.517593,6.218600,False,False,False,False,...,False,False,False,True,False,0,1,0,1,0
4,3303,9.143346,4.025352,3.526361,9.210540,7.496652,False,False,False,False,...,False,False,True,False,False,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25268,149835,10.396444,0.693147,0.693147,0.693147,3.465736,False,False,True,False,...,False,False,True,False,False,0,0,0,1,0
25269,1025,10.360976,0.693147,0.693147,0.693147,0.693147,False,False,False,False,...,False,False,True,False,False,0,1,0,1,0
25270,137113,10.396475,0.693147,0.693147,3.737670,4.406719,False,False,False,False,...,False,False,False,False,True,0,1,0,0,0
25271,269626,10.396506,0.693147,0.693147,0.693147,0.693147,False,False,True,False,...,False,False,False,False,True,0,0,0,0,0


In [24]:
# Normalizar los datos

# Crear objeto MinMaxScaler
scaler = MinMaxScaler()
# Ajustar y transformar solo esas columnas
final_df[num_list] = scaler.fit_transform(final_df[num_list])

final_df

,asn,org_id,ix_count,fac_count,info_prefixes4,info_prefixes6,policy_general_,policy_general_No,policy_general_Open,policy_general_Restrictive,...,info_ratio_Heavy Inbound,info_ratio_Heavy Outbound,info_ratio_Mostly Inbound,info_ratio_Mostly Outbound,info_ratio_Not Disclosed,policy_ratio,info_unicast,info_multicast,info_ipv6,info_never_via_route_servers
0,4436,0.855404,0.000000,0.000000,0.877352,0.758770,False,False,False,True,...,False,False,False,False,False,1,1,0,1,0
1,20940,0.153858,0.915669,0.753035,0.662966,0.697039,False,False,True,False,...,False,True,False,False,False,0,1,0,1,0
2,31800,0.172930,0.430313,0.000000,0.069827,0.000000,False,False,True,False,...,True,False,False,False,False,0,1,0,0,0
3,22822,0.178623,0.776295,0.606546,0.596268,0.492234,False,False,False,False,...,False,False,False,True,False,0,1,0,1,0
4,3303,0.860915,0.670022,0.475911,0.649075,0.606089,False,False,False,False,...,False,False,True,False,False,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25268,149835,0.999990,0.000000,0.000000,0.000000,0.246996,False,False,True,False,...,False,False,True,False,False,0,0,0,1,0
25269,1025,0.996053,0.000000,0.000000,0.000000,0.000000,False,False,False,False,...,False,False,True,False,False,0,1,0,1,0
25270,137113,0.999993,0.000000,0.000000,0.232010,0.330823,False,False,False,False,...,False,False,False,False,True,0,1,0,0,0
25271,269626,0.999997,0.000000,0.000000,0.000000,0.000000,False,False,True,False,...,False,False,False,False,True,0,0,0,0,0


In [38]:
for pref in categorical_list:

    # Filtramos las columnas que empiezan con ese prefijo
    columnas_con_prefijo = final_df.filter(like=pref).columns
    
    # Convertimos cada una de esas columnas a tipo entero
    for columna in columnas_con_prefijo:
        final_df[columna] = final_df[columna].astype(int)
final_df

,asn,org_id,ix_count,fac_count,info_prefixes4,info_prefixes6,policy_general_,policy_general_No,policy_general_Open,policy_general_Restrictive,...,info_ratio_Heavy Inbound,info_ratio_Heavy Outbound,info_ratio_Mostly Inbound,info_ratio_Mostly Outbound,info_ratio_Not Disclosed,policy_ratio,info_unicast,info_multicast,info_ipv6,info_never_via_route_servers
0,4436,0.855404,0.000000,0.000000,0.877352,0.758770,0,0,0,1,...,0,0,0,0,0,1,1,0,1,0
1,20940,0.153858,0.915669,0.753035,0.662966,0.697039,0,0,1,0,...,0,1,0,0,0,0,1,0,1,0
2,31800,0.172930,0.430313,0.000000,0.069827,0.000000,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0
3,22822,0.178623,0.776295,0.606546,0.596268,0.492234,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
4,3303,0.860915,0.670022,0.475911,0.649075,0.606089,0,0,0,0,...,0,0,1,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25268,149835,0.999990,0.000000,0.000000,0.000000,0.246996,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
25269,1025,0.996053,0.000000,0.000000,0.000000,0.000000,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
25270,137113,0.999993,0.000000,0.000000,0.232010,0.330823,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
25271,269626,0.999997,0.000000,0.000000,0.000000,0.000000,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [39]:
final_df.columns.values[0] = "ASN"  # Cambia el nombre de la primera columna
# Guardar el DataFrame final en un archivo CSV
final_df.to_csv('data/node_feature_mio.csv', index=False)


In [ ]:

# # Inicializamos una lista para guardar las filas del DataFrame
# data_rows = []

# # Recorrer cada ASN en los datos
# for asn in data:
#     row = {}  # Diccionario para cada fila
    
#     # Procesamos los atributos numéricos
#     for attr in num_list:
#         row[attr] = asn.get(attr, None)  # Obtén el valor o None si no existe

#     # Procesamos los atributos categóricos
#     for attr in categorical_list:
#         # Hacer one-hot encoding: para cada valor único, poner 1 si es el valor actual, 0 si no
#         uniques = set()
#         for asn in data:
#             value = asn.get(attr)
#             uniques.add(value)

#         for value in uniques:
#             row[f'{attr}_{value}'] = 1 if asn.get(attr) == value else 0
    
#     # Procesamos los valores booleanos
#     for attr in boolean_list:
#         row[attr] = asn.get(attr, False)  # Aseguramos que por defecto sea False si no hay valor
    
#     # Agregar la fila a la lista de filas
#     data_rows.append(row)

# # Crear el DataFrame
# df = pd.DataFrame(data_rows)

# # Mostrar el DataFrame
# print(df)


KeyboardInterrupt: 